This notebook generates ROCAUC graphs for the results/eval section

In [35]:
#imports
import numpy as np
from google.colab import drive
import os
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pickle

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/cs224w_project')

Mounted at /content/drive


In [36]:
def best_models(results):
    """
    Get best models by type

    Args:
        results (list of dict): List of dictionaries containing model results.
        topic (str): Topic for the table title.
        decade (int): Decade for the table title.

    Returns:
        list of dict: List of dictionaries containing best models.
    """
    # Organize results by regression model
    reg_model_tables = {}
    for result in results:
        reg_model = result['reg_model']
        if reg_model not in reg_model_tables:
            reg_model_tables[reg_model] = []
        reg_model_tables[reg_model].append(result)

    final_best_models = []
    for reg_model, models in reg_model_tables.items():
        # Group models by their model type
        grouped_by_model = {}
        for model in models:
            model_name = model['model']
            if model_name not in grouped_by_model:
                grouped_by_model[model_name] = []
            grouped_by_model[model_name].append(model)

        # Select the best result per model type
        final_best_models += [
            max(group, key=lambda x: x['auc_val']) for group in grouped_by_model.values()
        ]

    return final_best_models

In [51]:
def create_ROC_AUC_graphs(topic, decade, reg_model_type, split="test"):
    """
    Create ROC AUC graphs for a given topic and decade. Only plots curves for the best model per model type. Saves graph as a file.

    Args:
        topic (str): Topic for the table title.
        decade (int): Decade for the table title.
        reg_model_type (str): Type of regression model (Random Forest or Logistic Regression).

    """
    plt.figure(figsize=(8, 6))

    # Define folder paths
    folder_path = f"karsen_redo/EVAL/{topic}_{decade}{decade + 10}"
    save_folder = "Writeup/ROC_AUC_Curves/"

    # Ensure the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Define file paths
    file_path1 = os.path.join(folder_path, 'results_hetero.pkl')
    file_path2 = os.path.join(folder_path, 'results_homo.pkl')
    file_path3 = os.path.join(folder_path, 'results_baseline.pkl')

    # Load data
    with open(file_path1, 'rb') as file:
        data1 = pickle.load(file)
    with open(file_path2, 'rb') as file:
        data2 = pickle.load(file)
    with open(file_path3, 'rb') as file:
        data3 = pickle.load(file)

    # Combine and process data
    data = data1 + data2 + data3
    data = best_models(data)
    print(data[0])

    # Plot ROC curves
    for i in range(len(data)):
        if data[i]['reg_model'] == reg_model_type:
            model_name = data[i]['model']
            fpr = data[i][f"fpr_{split}"]
            tpr = data[i][f"tpr_{split}"]
            roc_auc = data[i][f"auc_{split}"]
            plt.plot(fpr, tpr, lw=2, label=f"{model_name} (AUC = {roc_auc:.3f})")

    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1, label='No Skill')

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"ROC Curve for {topic.title()} Dataset, {reg_model_type} Evaluation")
    plt.legend(loc='lower right')
    plt.grid()

    # Save the plot
    save_path = os.path.join(save_folder, f"{topic}_{decade}_{reg_model_type}_ROC_AUC.png")
    plt.savefig(save_path)
    plt.close()

    print(f"ROC AUC curve saved to {save_path}")


In [50]:
#Generating graphs for 1960s
for topic in ['labor', 'civil-rights', 'macro']:
  for reg_model in ['Logistic Regression', 'Random Forest']:
      decade = 60
      create_ROC_AUC_graphs(topic, decade, reg_model)

{'model': 'Heterogeneous SAGEConv', 'hyperparameters': {'alpha': 10, 'lr': 0.005, 'hidden size': 128, 'output size': 128, 'layers': 1, 'temperature': 0.08}, 'fpr_val': array([0.        , 0.61764706, 1.        ]), 'tpr_val': array([0.        , 0.78651685, 1.        ]), 'fpr_test': array([0.        , 0.26666667, 1.        ]), 'tpr_test': array([0.        , 0.70212766, 1.        ]), 'reg_model': 'Logistic Regression', 'val_accuracy': 0.6747967479674797, 'test_accuracy': 0.7096774193548387, 'auc_val': 0.6715135492399208, 'auc_test': 0.7546099290780142, 'report_val': '              precision    recall  f1-score   support\n\n         0.0       0.41      0.38      0.39        34\n         1.0       0.77      0.79      0.78        89\n\n    accuracy                           0.67       123\n   macro avg       0.59      0.58      0.59       123\nweighted avg       0.67      0.67      0.67       123\n', 'report_test': '              precision    recall  f1-score   support\n\n         0.0       0